## Analyzing CIA Factbook Data Using SQL

For this project we will work with data from the CIA World Factbook, which  collect statistics about all of the countries on earth, some of the demographic information are the next:
* population: the global population
* population_growth: the annual population growth rate, in percentage
* area: the total land and water area

the aim of the project is to use SQL in jupyter notebook to analyze data from this database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

This database has two tables but we will use facts name

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type="table";

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Overview Data

We will write a query in order to return the first five rows of the facts table

In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


the next are the descriptions for some of the columns:
* name — the name of the country.
* area— the country's total area (both land and water).
* area_land — the country's land area in square kilometers.
* area_water — the country's waterarea in square kilometers.
* population — the country's population.
* population_growth— the country's population growth as a percentage.
* birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
* death_rate — the country's death rate, or the number of death per year per 1,000 people.

## Summary statistics

Now let´s calculate de min population, max population, min population growth, max population growth

In [4]:
%%sql
SELECT MIN(population) AS "Min population",
       MAX(population) AS "Max population",
       MIN(population_growth) AS "Min population growth",
       MAX(population_growth) AS "Max population growth"
  FROM facts;

Done.


Min population,Max population,Min population growth,Max population growth
0,7256490011,0.0,4.02


As we can see the min population is 0 and the max one is 7.2 billion.
in the next step we will explore which countries these data corresponds

## Outliers exploration

First let´s figure out what it the country with 0 population

In [5]:
%%sql
SELECT *
  FROM facts
 WHERE population==(SELECT MIN(population)
                      FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


There is a row of Antarctica this explain population 0

In [6]:
%%sql
SELECT *
  FROM facts
 WHERE population==(SELECT MAX(population)
                      FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


The max population corresponds to World which explain 7,2 billion

## Recompute summary statistics

Now we will Recompute the summary statistics we found earlier while excluding the row for the whole world.

In [7]:
%%sql
SELECT MIN(population) AS "Min population",
       MAX(population) AS "Max population",
       MIN(population_growth) AS "Min population growth",
       MAX(population_growth) AS "Max population growth"
  FROM facts
 WHERE name <> "World";

Done.


Min population,Max population,Min population growth,Max population growth
0,1367485388,0.0,4.02


There is a country that with a poblation of 1.3 billion, let's to find it

In [8]:
%%sql
SELECT MAX(population) AS "max population",name
  FROM facts
 WHERE name <> "World";

Done.


max population,name
1367485388,China


country is the country with the max population on the world

## Exploring Average Population and Area

we will find the average population adn area but excepting the world data

In [9]:
%%sql
SELECT AVG(population) AS "Average population",
       AVG(area) AS "Average Area "
  FROM facts
 WHERE name <> "World";

Done.


Average population,Average Area
32242666.56846473,555093.546184739


Average population on the world is 32.242.666 and average area is 555.093 

## Finding Densely Populated Countries

Now let´s find the countries most density on the world writing a query that follow the next criteria:
* The population is above average.
* The area is below average.

In [10]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                       WHERE name <> "World") 

         AND area < (SELECT AVG(area)
                       FROM facts
                      WHERE name <> "World")
ORDER BY population DESC;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


we can see that there are 14 countries with the highest density

## Top 10 Countries with the highest population/area ratio

We will find the density for each country but using population/ area ratio which whows quantity of people per kilometer area

In [11]:
%%sql
SELECT name,(SELECT population / area) AS "population density"                      
FROM facts
WHERE "population density" <> "None"
ORDER BY "population density" DESC
LIMIT 10;

Done.


name,population density
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299


Macau is the most density country on the world with 21.168 people per kilometer.
This list is totally different to we found before since we use a a different criteria, in this case we use a ratio. 

## Top 10 country with the highest population growth rate

In [12]:
%%sql
  SELECT *
    FROM facts
ORDER BY "population_growth" DESC
LIMIT 10;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39
27,uv,Burkina Faso,274200,273800,400,18931686,3.03,42.03,11.72,0.0
109,ml,Mali,1240192,1220190,20002,16955536,2.98,44.99,12.89,2.26
219,cw,Cook Islands,236,236,0,9838,2.95,14.33,8.03,None
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62


South Sudan is the country with the highest population growth rate with 4.02 % growth.